In [9]:
import cv2 as cv
import numpy as np
import time

In [2]:
net = cv.dnn.readNetFromDarknet('custom-yolov4-tiny-detector-test.cfg', 'yolov4-tiny-detector_10000.weights')
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)

In [3]:
ln = net.getLayerNames()
print(len(ln), ln)

78 ['conv_0', 'bn_0', 'relu_1', 'conv_1', 'bn_1', 'relu_2', 'conv_2', 'bn_2', 'relu_3', 'slice_3', 'conv_4', 'bn_4', 'relu_5', 'conv_5', 'bn_5', 'relu_6', 'concat_6', 'conv_7', 'bn_7', 'relu_8', 'concat_8', 'pool_9', 'conv_10', 'bn_10', 'relu_11', 'slice_11', 'conv_12', 'bn_12', 'relu_13', 'conv_13', 'bn_13', 'relu_14', 'concat_14', 'conv_15', 'bn_15', 'relu_16', 'concat_16', 'pool_17', 'conv_18', 'bn_18', 'relu_19', 'slice_19', 'conv_20', 'bn_20', 'relu_21', 'conv_21', 'bn_21', 'relu_22', 'concat_22', 'conv_23', 'bn_23', 'relu_24', 'concat_24', 'pool_25', 'conv_26', 'bn_26', 'relu_27', 'conv_27', 'bn_27', 'relu_28', 'conv_28', 'bn_28', 'relu_29', 'conv_29', 'permute_30', 'yolo_30', 'identity_31', 'conv_32', 'bn_32', 'relu_33', 'upsample_33', 'concat_34', 'conv_35', 'bn_35', 'relu_36', 'conv_36', 'permute_37', 'yolo_37']


In [6]:
img = cv.imread("Picture 515.jpg", cv.IMREAD_UNCHANGED)

In [7]:
blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)

In [11]:
net.setInput(blob)
t0 = time.time()
outputs = net.forward(ln)
t = time.time()
print('time=', t-t0)

time= 0.13297772407531738


In [12]:
print(len(outputs))
for out in outputs:
    print(out.shape)

78
(1, 32, 208, 208)
(1, 32, 208, 208)
(1, 32, 208, 208)
(1, 64, 104, 104)
(1, 64, 104, 104)
(1, 64, 104, 104)
(1, 64, 104, 104)
(1, 64, 104, 104)
(1, 64, 104, 104)
(1, 32, 104, 104)
(1, 32, 104, 104)
(1, 32, 104, 104)
(1, 32, 104, 104)
(1, 32, 104, 104)
(1, 32, 104, 104)
(1, 32, 104, 104)
(1, 64, 104, 104)
(1, 64, 104, 104)
(1, 64, 104, 104)
(1, 64, 104, 104)
(1, 128, 104, 104)
(1, 128, 52, 52)
(1, 128, 52, 52)
(1, 128, 52, 52)
(1, 128, 52, 52)
(1, 64, 52, 52)
(1, 64, 52, 52)
(1, 64, 52, 52)
(1, 64, 52, 52)
(1, 64, 52, 52)
(1, 64, 52, 52)
(1, 64, 52, 52)
(1, 128, 52, 52)
(1, 128, 52, 52)
(1, 128, 52, 52)
(1, 128, 52, 52)
(1, 256, 52, 52)
(1, 256, 26, 26)
(1, 256, 26, 26)
(1, 256, 26, 26)
(1, 256, 26, 26)
(1, 128, 26, 26)
(1, 128, 26, 26)
(1, 128, 26, 26)
(1, 128, 26, 26)
(1, 128, 26, 26)
(1, 128, 26, 26)
(1, 128, 26, 26)
(1, 256, 26, 26)
(1, 256, 26, 26)
(1, 256, 26, 26)
(1, 256, 26, 26)
(1, 512, 26, 26)
(1, 512, 13, 13)
(1, 512, 13, 13)
(1, 512, 13, 13)
(1, 512, 13, 13)
(1, 256, 13, 

In [ ]:

def trackbar2(x):
    confidence = x/100
    r = r0.copy()
    for output in np.vstack(outputs):
        if output[4] > confidence:
            x, y, w, h = output[:4]
            p0 = int((x-w/2)*416), int((y-h/2)*416)
            p1 = int((x+w/2)*416), int((y+h/2)*416)
            cv.rectangle(r, p0, p1, 1, 1)
    cv.imshow('blob', r)
    text = f'Bbox confidence={confidence}'
    cv.displayOverlay('blob', text)

r0 = blob[0, 0, :, :]
r = r0.copy()
cv.imshow('blob', r)
cv.createTrackbar('confidence', 'blob', 50, 101, trackbar2)
trackbar2(50)

boxes = []
confidences = []
classIDs = []
h, w = img.shape[:2]

for output in outputs:
    for detection in output:
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > 0.5:
            box = detection[:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype("int")
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            box = [x, y, int(width), int(height)]
            boxes.append(box)
            confidences.append(float(confidence))
            classIDs.append(classID)

indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
if len(indices) > 0:
    for i in indices.flatten():
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        color = [int(c) for c in colors[classIDs[i]]]
        cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
        cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
